In [1]:
import sqlite3 
import pandas as pd
#import pandas_profiling as pp
import numpy as np

import statsmodels.api as sm

import seaborn as sns

sns.set_style("whitegrid")


In [2]:
conn = sqlite3.connect('traffic.db')
c = conn.cursor()

In [ ]:
sql_master = """SELECT * FROM sqlite_master"""
pd.read_sql_query(sql_master, conn)

In [ ]:
### sql_master = """SELECT sql FROM sqlite_master"""
pd.read_sql_query(sql_master, conn)

In [ ]:
sql_all = "SELECT * FROM traffic"
all_df = pd.read_sql_query(sql_all, conn)
all_df.head()

In [ ]:
all_df.columns

In [ ]:
#pp.ProfileReport(all_df)

In [ ]:
all_df.describe()

In [ ]:
all_df['Violation Type'].value_counts()

In [ ]:
grouped_race_violation = all_df.groupby(['Race', 'Violation Type']).size().astype(float)

In [ ]:
all_df.groupby(['Race']).size()

In [ ]:
all_df['Arrest Type'].value_counts()

In [ ]:
sum(grouped_race_violation['WHITE'])

In [ ]:
grouped_normalize = grouped_race_violation.copy()

for race in grouped_race_violation.index.levels[0]:
    race_viol = sum(grouped_race_violation[race])
    for viol in grouped_race_violation.index.levels[1]:
        try:
            grouped_normalize[race][viol] = grouped_race_violation[race][viol] / float(race_viol)
        except:
            pass

In [ ]:
grouped_normalize

In [ ]:
all_df.groupby(['Race', 'Arrest Type']).size().astype(float)

In [ ]:
all_df['Arrest Type'].value_counts(True)

In [ ]:
len(all_df)

In [ ]:
descrips = all_df.Description.value_counts()

In [ ]:
descrips

In [ ]:
np.percentile(descrips.values, 90, axis=0)

In [9]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
a = sorted(descrips.values)
cumsum_descrips = [sum(a[:i]) for i in range(len(a))]
#cumsum_descrips

In [ ]:
x_percentile = (np.array(range(len(a)))+1)/len(a)


plt.plot(x_percentile, cumsum_descrips)

In [ ]:
all_df.Description.value_counts().index[1]

In [ ]:
viol_type = 'FAILURE TO DISPLAY REGISTRATION CARD UPON DEMAND BY POLICE OFFICER'

all_df[all_df['Description'].isin([viol_type])].groupby('Race').size()

In [ ]:
sql_all = '''
SELECT * FROM traffic
WHERE race = 'BLACK'
ORDER BY RANDOM() 
LIMIT 5000
'''
df_black = pd.read_sql_query(sql_all, conn)
df_black.shape

In [ ]:
sql_all = '''
SELECT * FROM traffic
WHERE race = 'HISPANIC'
ORDER BY RANDOM() 
LIMIT 5000
'''
df_hispanic = pd.read_sql_query(sql_all, conn)
df_hispanic.shape

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
black_citation_sample = (df_black['Violation Type'] == 'Citation')*1
mu_black = sum(black_citation_sample) / len(black_citation_sample)

In [ ]:
hispanic_citation_sample = (df_hispanic['Violation Type'] == 'Citation')*1
mu_hispanic = sum(hispanic_citation_sample) / len(hispanic_citation_sample)

In [ ]:
ttest_ind(black_citation_sample, hispanic_citation_sample, equal_var=False)

In [ ]:
def sample_from_traffic(n_sample, connection, race=None, gender=None, show_SQL=True):
    """Returns a random sample from the specified sqlite db connection, from the table called traffic.
    Order is randomized.
    Can specify a race to filter on.
    Prints the equivalent SQL query.
    Returns the dataframe result of the SQL query"""
    var_count = 0
    if race:
        condition = "WHERE race ='{}'".format(race.upper())
        var_count += 1
    else:
        condition = "--"
    if gender:
        if var_count > 0:
            condition = condition + """
            AND Gender = '{}'""".format(gender.upper()[0])
        else:
            condition = "WHERE gender ='{}'".format(gender.upper()[0])
    sql_str = '''
        SELECT * FROM traffic
        {0}
        ORDER BY RANDOM() 
        LIMIT {1}
        '''.format(condition, n_sample)
    if show_SQL: print(sql_str)
    df = pd.read_sql_query(sql_str, connection)
    return df

In [3]:
def sample_from_traffic_2(n_sample, connection, filters={}, show_SQL=True):
    """Returns a random sample from the specified sqlite db connection, from the table called traffic.
    Order is randomized.
    Can specify a race to filter on.
    Prints the equivalent SQL query.
    Returns the dataframe result of the SQL query"""
    var_count = 0
    condition = "--"
    for k, v in filters.items():
        if var_count==0:
            if type(v)!=str:
                condition = "WHERE {0} in {1}".format(k, tuple(v))
            else:
                condition = "WHERE {0} = '{1}'".format(k, v)
        else:
            if type(v)!=str:
                condition = condition + """
                AND {0} in {1}""".format(k, tuple(v))
            else:
                condition = condition + """
                AND {0} = '{1}'""".format(k, v)
        
        var_count += 1

    sql_str = '''
        SELECT * FROM traffic
        {0}
        ORDER BY RANDOM() 
        LIMIT {1}
        '''.format(condition, n_sample)
    if show_SQL: print(sql_str)
    df = pd.read_sql_query(sql_str, connection)
    return df

In [ ]:
df_white = sample_from_traffic(5000, conn, 'White')
df_black = sample_from_traffic(5000, conn, 'Black')
df_hispanic = sample_from_traffic(5000, conn, 'Hispanic')
df_asian = sample_from_traffic(5000, conn, 'Asian')
df_other = sample_from_traffic(5000, conn, 'other')

df_race_sample = pd.concat([df_white, df_asian, df_black, df_hispanic, df_other])


X = pd.get_dummies(df_race_sample['Race'], prefix='Race')
X = X.drop(columns='Race_WHITE')
y = (df_race_sample['Violation Type'] == 'Citation')*1
y.columns = ['Citation']

X = sm.add_constant(X)

model_linreg = sm.OLS(y, X).fit()
model_linreg.summary()

In [ ]:
df_test = sample_from_traffic(5000, conn)

In [ ]:
df_test.Race.value_counts()

In [ ]:
pd.to_datetime(all_df['Date Of Stop'], format='%m/%d/%Y').describe()

In [ ]:
import datetime

In [ ]:
fig = plt.figure(figsize=(16, 6))
plt.hist(pd.to_datetime(all_df['Date Of Stop'], format='%m/%d/%Y'), bins=88);
#plt.xlim(datetime.date(2016,1,1))

In [ ]:
all_df.Color.value_counts()

In [ ]:
# Linear regression of Race versus Citation

df_white = sample_from_traffic(5000, conn, 'White')
df_black = sample_from_traffic(5000, conn, 'Black')
df_hispanic = sample_from_traffic(5000, conn, 'Hispanic')
df_asian = sample_from_traffic(5000, conn, 'Asian')
df_other = sample_from_traffic(5000, conn, 'other')

df_race_sample = pd.concat([df_white, df_asian, df_black, df_hispanic, df_other])


X = pd.get_dummies(df_race_sample['Race'], prefix='Race')
X = X.drop(columns='Race_WHITE')
y = (df_race_sample['Violation Type'] == 'Citation')*1
y.columns = ['Citation']

X = sm.add_constant(X)

model_logit = sm.Logit(y, X).fit()
model_logit.summary()

In [ ]:
# 2 prop z-test on white versus black citations

from statsmodels.stats.proportion import proportions_ztest


n_sim = 100

mu_white_list = []
mu_black_list = []
z_list = []
p_list = []

for i in range(n_sim):
    n_sample = 5000

    df_white = sample_from_traffic(n_sample, conn, 'White', show_SQL=False)
    df_black = sample_from_traffic(n_sample, conn, 'Black', show_SQL=False)

    white_cited = sum(df_white['Violation Type'] == 'Citation')
    black_cited = sum(df_black['Violation Type'] == 'Citation')

    mu_white_list.append(white_cited / n_sample)
    mu_black_list.append(black_cited / n_sample)
    
    count = np.array([white_cited, black_cited])
    nobs = np.array([n_sample, n_sample])
    z,p = proportions_ztest(count, nobs, value=0, alternative='two-sided')
    z_list.append(z)
    p_list.append(p)
    


fig = plt.figure(figsize=(16,10))


sns.distplot(mu_white_list, label="White Proportion Cited")
sns.distplot(mu_black_list, label="Black Proportion Cited")
plt.title('Montgomery County Traffic Stops \n Proportion Issued Citation - White vs. Black', fontsize=20)

plt.xlabel('Proportion of Sample Issued Citation', fontsize=16)
plt.ylabel('Frequency from {} simulations'.format(n_sim), fontsize=16)
plt.legend();


In [ ]:
# 2 prop z-test on white versus black warnings

from statsmodels.stats.proportion import proportions_ztest

#n_sim is repeating the whole sampling process of 5000, 100 times
n_sim = 100

mu_white_list = []
mu_black_list = []
z_list = []
p_list = []

for i in range(n_sim):
    n_sample = 5000

    
    descrip = "FAILURE TO DISPLAY REGISTRATION CARD UPON DEMAND BY POLICE OFFICER"
    
    df_white = sample_from_traffic_2(n_sample, conn, filters={'race':'WHITE', 'Description':descrip}, show_SQL=False)
    df_black = sample_from_traffic_2(n_sample, conn, filters={'race':'BLACK', 'Description':descrip}, show_SQL=False)

    white_cited = sum(df_white['Violation Type'] == 'Warning')
    black_cited = sum(df_black['Violation Type'] == 'Warning')

    mu_white_list.append(white_cited / n_sample)
    mu_black_list.append(black_cited / n_sample)
    
    count = np.array([white_cited, black_cited])
    nobs = np.array([n_sample, n_sample])
    z,p = proportions_ztest(count, nobs, value=0, alternative='two-sided')
    z_list.append(z)
    p_list.append(p)
    


fig = plt.figure(figsize=(16,10))


sns.distplot(mu_white_list, label="White Proportion Given Warning")
sns.distplot(mu_black_list, label="Black Proportion Given Warning")
plt.title('Montgomery County Traffic Stops \n Did Not Present Registration to Officer \n Proportion Given Warning - White vs. Black', fontsize=20)

plt.xlabel('Proportion of Sample Given Warning', fontsize=16)
plt.ylabel('Frequency from {} simulations'.format(n_sim), fontsize=16)
plt.legend();


In [4]:
# 2 prop z-test on white versus black warnings
# we are running a z-test instead of a t-test because we're looking at the proportions of sample 
#given a warning rather than the number of people

from statsmodels.stats.proportion import proportions_ztest

#n_sim is repeating the whole sampling process of 5000 people for each category, 100 times
n_sim = 100


mu_black_male_list = []
mu_black_female_list = []
mu_white_male_list = []
mu_white_female_list = []

z_list = []
p_list = []

for i in range(n_sim):
    n_sample = 5000

    df_black_male = sample_from_traffic_2(n_sample, conn, filters={'race':'BLACK','gender':'M'}, show_SQL=False)
    df_black_female = sample_from_traffic_2(n_sample, conn, filters={'race':'BLACK', 'gender':'F'}, show_SQL=False)
    df_white_male = sample_from_traffic_2(n_sample, conn, filters={'race':'BLACK', 'gender':'M'}, show_SQL=False)
    df_white_female = sample_from_traffic_2(n_sample, conn, filters={'race':'BLACK', 'gender':'F'}, show_SQL=False)


    black_male_cited = sum(df_black_male['Violation Type'] == 'Citation')
    black_female_cited = sum(df_black_female['Violation Type'] == 'Citation')
    white_male_cited = sum(df_white_male['Violation Type'] == 'Citation')
    white_female_cited = sum(df_white_female['Violation Type'] == 'Citation')

    mu_black_male_list.append(black_male_cited / n_sample)
    mu_black_female_list.append(black_female_cited / n_sample)
    mu_white_male_list.append(white_male_cited / n_sample)
    mu_white_female_list.append(white_female_cited / n_sample)
    
    nobs = np.array([n_sample, n_sample])
    
    z1,p1 = proportions_ztest(np.array([black_male_cited, black_female_cited]), nobs, value=0, alternative='two-sided')
    z2,p2 = proportions_ztest(np.array([black_male_cited, white_female_cited]), nobs, value=0, alternative='two-sided')
    z3,p3 = proportions_ztest(np.array([white_female_cited, black_female_cited]), nobs, value=0, alternative='two-sided')
    z4,p4 = proportions_ztest(np.array([white_male_cited, white_female_cited]), nobs, value=0, alternative='two-sided')

    z_list.append([z1, z2, z3, z4])
    p_list.append([p1, p2, p3, p4])
    



In [ ]:
p1

In [ ]:
p2

In [ ]:
p3

In [ ]:
p4

In [ ]:
z1_list =[x[0] for x in z_list]
z1_list

In [ ]:
p_list

In [12]:
avg_mu_black_male_list= sum(mu_black_male_list)/len(mu_black_male_list)
avg_mu_black_female_list= sum(mu_black_female_list)/len(mu_black_female_list)
avg_mu_white_male_list= sum(mu_white_male_list)/len(mu_white_male_list)
avg_mu_white_female_list= sum(mu_white_female_list)/len(mu_white_female_list)

In [20]:
avg_mu_black_male_list

0.48970400000000003

In [21]:
avg_mu_black_female_list

0.4230000000000001

In [22]:
avg_mu_white_male_list

0.4905380000000001

In [23]:
avg_mu_white_female_list

0.4215379999999999

In [19]:
#Black Male vs. Black Female Citation Rate
fig = plt.figure(figsize=(16,10))

sns.set()
cols = ['price']
independent_variables = ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement_calc', 'bedrooms', 'bathrooms', 'floors', 'condition', 'grade', 'yr_renovated', 'view']
independent_variables.sort()
cols.extend(independent_variables)
sns.pairplot(df_cleaned[cols], size = 3.5)
plt.show();

sns.barplot(avg_mu_black_male_list, avg_mu_black_female_list, label="Black Males vs. Females: Proportion Given Citation")


TypeError: 'float' object is not iterable

<Figure size 1152x720 with 0 Axes>

In [ ]:
#Black Male vs. White Male Citation Rate
fig = plt.figure(figsize=(16,10))

sns.distplot(mu_black_male_list, label="Black Male: Proportion Given Citation")
sns.distplot(mu_white_female_list, label="White Male: Proportion Given Citation")


plt.title('Montgomery County Traffic Stops \n Proportion Given Citation \n Black Male vs. White Males', fontsize=20)

plt.xlabel('Proportion of Sample Given Citation', fontsize=16)
plt.ylabel('Frequency from {} simulations'.format(n_sim), fontsize=16)
plt.legend();

In [ ]:
#White Male vs. White Female Citation Rate
fig = plt.figure(figsize=(16,10))

sns.distplot(mu_white_male_list, label="White Males: Proportion Given Citation")
sns.distplot(mu_white_female_list, label="White Females: Proportion Given Citation")


plt.title('Montgomery County Traffic Stops \n Proportion Given Citation \n White Male vs. White Females', fontsize=20)

plt.xlabel('Proportion of Sample Given Citation', fontsize=16)
plt.ylabel('Frequency from {} simulations'.format(n_sim), fontsize=16)
plt.legend();

In [ ]:
#White Female vs. Black Female Citation Rate
fig = plt.figure(figsize=(16,10))

sns.distplot(mu_black_female_list, label="Black Females: Proportion Given Citation")
sns.distplot(mu_white_female_list, label="White Females: Proportion Given Citation")


plt.title('Montgomery County Traffic Stops \n Proportion Given Citation \n Black Females vs. White Females', fontsize=20)

plt.xlabel('Proportion of Sample Given Citation', fontsize=16)
plt.ylabel('Frequency from {} simulations'.format(n_sim), fontsize=16)
plt.legend();